In [4]:
import datetime
from ray_of_slurm import RayCluster

cluster = RayCluster(
    cpus_per_task=4,
    ntasks_per_node=1,
    nodes=2,
    job_name="notebookslurm",
    account="nickp",
    time=datetime.timedelta(minutes=10)
)


In [5]:
cluster.sbatch("python minimal_ray.py", shell="/bin/bash")

TypeError: Slurm.srun() got an unexpected keyword argument 'shell'

In [3]:
print(cluster.script())

#!/bin/sh

#SBATCH --account             nickp
#SBATCH --cpus-per-task       4
#SBATCH --job-name            notebookslurm
#SBATCH --nodes               2
#SBATCH --ntasks-per-node     1
#SBATCH --time                0-00:10:00

source /home/posit0002/ray-of-slurm/.venv/bin/activate
echo \$pwd
head_node=\$(hostname)
head_node_ip=\$(hostname --ip-address)
if [[ "\$head_node_ip" == *" "* ]]; then
IFS=' ' read -ra ADDR <<<"\$head_node_ip"
if [[ \${#ADDR[0]} -gt 16 ]]; then
head_node_ip=\${ADDR[1]}
else
head_node_ip=\${ADDR[0]}
fi
fi
port=6379
echo "STARTING HEAD at \$head_node"
echo "Head node IP: \$head_node_ip"
srun --nodes=1 --ntasks=1 -w \$head_node start_head.sh \$head_node_ip &
sleep 10
worker_num=\$((\$SLURM_JOB_NUM_NODES - 1))
srun -n \$worker_num --nodes=\$worker_num --ntasks-per-node=1 --exclude \$head_node start_worker.sh \$head_node_ip:\$port &
sleep 5
python minimal_ray.py



In [6]:
cluster.srun("python minimal_ray.py")

Traceback (most recent call last):
  File "/home/posit0002/ray-of-slurm/minimal_ray.py", line 6, in <module>
Traceback (most recent call last):
  File "/home/posit0002/ray-of-slurm/minimal_ray.py", line 6, in <module>
srun: error: all-dy-rstudio-5: task 0: Exited with exit code 1
srun: error: all-dy-rstudio-6: task 1: Exited with exit code 1
    import ray
ModuleNotFoundError: No module named 'ray'
    import ray
ModuleNotFoundError: No module named 'ray'


CalledProcessError: Command 'srun --account=nickp --cpus-per-task=4 --job-name=notebookslurm --nodes=2 --ntasks-per-node=1 --time=0-00:10:00 python minimal_ray.py' returned non-zero exit status 1.